In [1]:
import numpy as np
# import joblib
# from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# import torchmetrics
import models
from utils import SoftLabelFbankCustom
import utils
from engine import train
from torchinfo import summary

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

d:\code\Py\Ai\IEEE comsoc\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:


eval_csv_path = "./data_csv/fold1_eval_soft_labels.csv"
train_csv_path = "./data_csv/fold1_train_soft_labels.csv"
csv_path = "./data/esc_class_labels_indices.csv"


# nerfing the batch size too
eval_loader = DataLoader(SoftLabelFbankCustom(eval_csv_path, 50),
                          batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
train_loader = DataLoader(SoftLabelFbankCustom(train_csv_path, 50),
                           batch_size=32, shuffle=False, num_workers=0, pin_memory=True)


# os.makedirs(config_dict['exp_dir'], exist_ok=True)

In [3]:
model = models.CRNN4(input_shape=(128, 512, 1), output_shape=50,
                 cnn_layers=2,kernels=(5, 3, 3), filters=(32, 64, 64), pool_sizes=(2, 2, 2),
                 cnn_dropout=(0.3, 0.4, 0.4),
                 rnn_layers=2, rnn_hidden=(64, 128),
                 rnn_dropout=(0.4, 0.2),

                fnn_layers=1, fnn_hidden=(512,), fnn_dropout=(0.4,),)
# damn that's a hard nerf


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.75)
loss_fn = nn.CrossEntropyLoss()
accuracy_multi_class = lambda pred, tar: (torch.argmax(pred, dim=1) == torch.argmax(tar, dim=1)).float().mean()
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
model = model.to(device)

summary(model)

Layer (type:depth-idx)                   Param #
CRNN4                                    --
├─ModuleList: 1-1                        --
│    └─Conv2d: 2-1                       832
│    └─ReLU: 2-2                         --
│    └─MaxPool2d: 2-3                    --
│    └─BatchNorm2d: 2-4                  64
│    └─Dropout: 2-5                      --
│    └─Conv2d: 2-6                       18,496
│    └─ReLU: 2-7                         --
│    └─MaxPool2d: 2-8                    --
│    └─BatchNorm2d: 2-9                  128
│    └─Dropout: 2-10                     --
├─ModuleList: 1-2                        --
│    └─GRU: 2-11                         405,888
│    └─LayerNorm: 2-12                   128
│    └─Dropout: 2-13                     --
│    └─GRU: 2-14                         74,496
│    └─LayerNorm: 2-15                   256
│    └─Dropout: 2-16                     --
├─Sequential: 1-3                        --
│    └─Linear: 2-17                      66,048
│    └

In [4]:
# check = torch.load("./checkpoints/2best_model_crnn4_18ep_highReg.pth")


# model.load_state_dict(check['model_state_dict'])
# optimizer.load_state_dict(check['optim_state_dict'])



In [26]:
# optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.1
# optimizer.param_groups[0]['weight_decay'] = optimizer.param_groups[0]['weight_decay'] * 100

In [8]:


train(model, train_loader, eval_loader, optimizer=optimizer, loss_fn=loss_fn, metric_fn=accuracy_multi_class, epochs=3, device=device, check_dir='./checkpoints', checkpoint_dict=check)

  0%|          | 0/3 [00:00<?, ?it/s]

starting training batches:
0 / 50
16 / 50
32 / 50
48 / 50
Epoch: 1 | train_loss: 0.8930 | train_metric: 0.7525 | test_loss: 2.4857 | test_metric: 0.3678


 33%|███▎      | 1/3 [00:16<00:33, 16.90s/it]

starting training batches:
0 / 50
16 / 50
32 / 50
48 / 50
Epoch: 2 | train_loss: 0.8690 | train_metric: 0.7550 | test_loss: 2.4667 | test_metric: 0.4207


 67%|██████▋   | 2/3 [00:37<00:19, 19.18s/it]

starting training batches:
0 / 50
16 / 50
32 / 50
48 / 50


100%|██████████| 3/3 [00:57<00:00, 19.11s/it]

Epoch: 3 | train_loss: 0.8096 | train_metric: 0.7769 | test_loss: 2.4164 | test_metric: 0.4351


{'train_loss': [4.126364107131958,
  3.9241414976119997,
  3.8631375217437744,
  3.790391216278076,
  3.7428553342819213,
  3.695086636543274,
  3.646302523612976,
  3.6389204597473146,
  3.5280529165267946,
  3.470722074508667,
  3.4276807928085327,
  3.319245705604553,
  3.238357214927673,
  3.1557249879837035,
  3.0188574504852297,
  2.7893743467330934,
  2.650936326980591,
  2.529330008029938,
  2.387311723232269,
  2.2624942374229433,
  2.157312316894531,
  2.085415630340576,
  1.9564237904548645,
  1.8550985717773438,
  1.8122423791885376,
  1.74238440990448,
  1.5940260899066925,
  1.5064444375038146,
  1.4063128328323364,
  1.3174234592914582,
  1.231454049348831,
  1.1157502245903015,
  1.0788812363147735,
  1.0248507285118102,
  0.9574160492420196,
  0.8929908657073975,
  0.8689860618114471,
  0.8096010333299637],
 'train_metric': [0.023749999701976776,
  0.036249998956918716,
  0.0456249974668026,
  0.04437499865889549,
  0.05249999836087227,
  0.06187499687075615,
  0.05999